In [2]:
class Book:

    def __init__(self, title, author, publication_year, available_copies):
        self.title = title
        self.author = author
        self.publication_year = publication_year
        self.available_copies = available_copies

    def __str__(self):
        return f"Title: {self.title}, Author: {self.author}, Publication Year: {self.publication_year}"
    
    def getTitle(self):
        return str(self.title)
    
    def setTitle(self, title):
        self.title = title


book1 = Book("To Kill a Mockingbird", "Harper Lee", 196, 24)
book2 = Book("1984", "George Orwell", 194, 13)
print(book1)
print(book2)

Title: To Kill a Mockingbird, Author: Harper Lee, Publication Year: 196
Title: 1984, Author: George Orwell, Publication Year: 194


In [ ]:
class person:

    def__init__(self, fname, lname):